In [12]:
pip install pandas "pyspark[connect]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 MB 63.0 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-4.0.0-py2.py3-none-any.whl size=434741299 sha256=d98ea67ca67f9b93da0721cc292cf5a713402f2df561cbc3eda769c2caa01c86
  Stored in directory: /Users/chenguo/Library/Caches/pip/wheels/2d/77/9b/12660be70f7f447940a0caede37ae208b2e0d1c8487dce52a6
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 4.0.1
    Uninstalling pyspark-4.0.1:
      Successfully uninstalled pyspark-4.0.1

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [38]:
import pyspark, sys
print("pyspark:", pyspark.__version__, "python:", sys.version)

pyspark: 4.0.0 python: 3.12.11 (main, Aug 27 2025, 14:00:03) [Clang 17.0.0 (clang-1700.0.13.5)]


In [39]:
from pyspark.sql import SparkSession

# Create a Spark session using Connect
spark = (
    SparkSession.builder
    # May need to do `sudo sh -c 'echo "127.0.0.1 spark-connect-svc.spark-operator.svc.cluster.local" >> /etc/hosts'`
    # For kubectl port-forward, k8s sets up a local socket on 127.0.0.1:15002 that tunnels traffic to the service spark-connect-svc 
    # inside the cluster. However, Spark Connect isn’t plain TCP; it’s gRPC/HTTP 2 and often TLS-aware (even if you didn’t configure full HTTPS).
    # During connection setup, the client identifies itself using the authority/hostname portion of the URI (sc://...).
    # If your notebook connects with `.remote("sc://127.0.0.1:15002")`, the client says “authority = 127.0.0.1”, but the server thinks 
    # its own name is `spark-connect-svc.spark-operator.svc.cluster.local`

    # .remote("sc://spark-connect-svc.spark-operator.svc.cluster.local:15002")
    .remote("sc://localhost:15002")
    .appName("LocalJupyterNotebook") # Not very useful for Spark Connect scenario
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true")
    .config("spark.dynamicAllocation.initialExecutors", "1")
    .config("spark.dynamicAllocation.minExecutors", "1")
    .config("spark.dynamicAllocation.maxExecutors", "8")
    .config("spark.default.parallelism", "1024")
    .config("spark.sql.shuffle.partitions", "1024")
    .getOrCreate()
)

# Test the connection
print(f"Spark version: {spark.version}")
print(f"Spark Connect session id: {spark.client._session_id}")
print(f"App Name: {spark.conf.get("spark.app.name")}")

Spark version: 4.0.0
Spark Connect session id: b86c0e60-c72e-4633-817f-0f0ee799032f
App Name: LocalJupyterNotebook


In [40]:
print(spark.conf.get("spark.dynamicAllocation.enabled", "not set"))
print(spark.conf.get("spark.dynamicAllocation.shuffleTracking.enabled", "not set"))
print(spark.conf.get("spark.dynamicAllocation.initialExecutors", "not set"))
print(spark.conf.get("spark.dynamicAllocation.minExecutors", "not set"))
print(spark.conf.get("spark.dynamicAllocation.maxExecutors", "not set"))
print(spark.conf.get("spark.default.parallelism", "not set"))
print(spark.conf.get("spark.sql.shuffle.partitions", "not set"))

true
true
1
1
50
1024
1024


In [41]:
df = spark.range(6)
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
+---+



In [34]:
from pyspark.sql.functions import rand
df = spark.range(0, 50_000_000, numPartitions=2000).withColumn("x", rand())
a  = df.repartition(2000, "id").groupBy((df.id % 500).alias("k")).count()
b  = df.repartition(2000, "id").groupBy((df.id % 500).alias("k")).avg("x")
a.join(b, "k").count()


KeyboardInterrupt


KeyboardInterrupt



In [9]:
spark.stop()